# Set-up

## Installing libraries and libcudnn8

In [1]:
import os

FILEID = "1h4FWB5fw7sBDCSM-EENK1UadqKSCqg24"

contents = os.listdir(os.getcwd())

if 'MI_EEG_ClassMeth' not in contents:
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
    !unzip MI_EEG_ClassMeth.zip
else:
    print("MI_EEG_ClassMeth already downloaded!")

!apt-get install --allow-change-held-packages libcudnn8=8.1.1.33-1+cuda11.2 -y
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases
!pip install mne
!pip install pickle5
!pip install gcpds.utils
!pip install scikeras[tensorflow]

MI_EEG_ClassMeth already downloaded!
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 91 not upgraded.
Need to get 421 MB of archives.
After this operation, 2621 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.1.1.33-1+cuda11.2 [421 MB]
Fetched 421 MB in 5s (81.0 MB/s)    
(Reading database ... 108827 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.0) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
(Reading database ... 108805 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.1.33-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.1.33-1+cu

## Import libraries

In [2]:
# freq filter 
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr

#EEG montage
from gcpds.utils.mne_handler import get_best_montage

# general
import numpy as np
from scipy.signal import resample
import pickle5 as pickle
import warnings
import mne
from time import time
warnings.filterwarnings('ignore')

# tensorlfow 
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, AveragePooling2D, BatchNormalization, Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import L1L2

# scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV,StratifiedShuffleSplit
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score

## Define functions

In [3]:
def kappa(y_true, y_pred):
    return cohen_kappa_score(np.argmax(y_true, axis = 1),np.argmax(y_pred, axis = 1))

## PAIN dataset

In [4]:
def load_PAIN(db,sbj,f_bank,vwt,new_fs):

    channels_names = np.array(['Fp1','Fp2',
                      'F3','F4','C3','C4','P3','P4','O1','O2','F7','F8',
                      'T7','T8','P7','P8','Fz','Cz','Pz','Oz',
                      'FC1','FC2','CP1','CP2','FC5','FC6','CP5','CP6',
                      'TP9','TP10','LE','RE','P1','P2','C1','C2',
                      'FT9','FT10','AF3','AF4','FC3','FC4','CP3','CP4','PO3','PO4',
                      'F5','F6','C5','C6','P5','P6','PO9','Iz','FT7','FT8',
                      'TP7','TP8','PO7','PO8','Fpz','PO10','CPz','POz',
                      'Ne','Ma','Ext','ECG'])
    
    with open('{}BMOP_Motor_S{}.pkl'.format(db,'0' + str(sbj) if sbj < 10 else sbj), 'rb') as f:
        data = pickle.load(f)
        
    X = data['X']  # trials, channels, time
    y = data['y']
    sex = data['sex'].ravel()
    age = data['age'].ravel()
    fs = float(data['fs'])
    
    tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)
    
    #Read electrode positions to load the best standard montage-MNE
    best_montages = get_best_montage(channels_names)
    montage = best_montages.iloc[0]['montage']
    no_channels = np.array(best_montages.iloc[0]['missings channels'])
    channels_to_remove = np.array([np.argwhere(channels_names==no)[0] for no in no_channels])[:,0]

    #Delete the missing channels the original array respecting the positions
    channels_names = np.delete(channels_names, channels_to_remove)
    X = np.delete(X, channels_to_remove, axis=1)

    #Number channels does not match with the dimension of X, 
    #thus the last channel is discarded because it has weird amplitudes
    X = X[:,:-1,:]

    info = mne.create_info(list(channels_names), sfreq=fs, ch_types="eeg")
    info.set_montage(montage)
    info

    event_id = {
        'pain/high':2,
        'resting':3,
        }

    events = [[i, 1, cls[0]] for i, cls in enumerate(y)]
    tmin = 0

    epochs = mne.EpochsArray(X, info, events=events, tmin=tmin, event_id=event_id)
    X = epochs.get_data()
    y = y-2
    X = np.squeeze(tf_repr.transform(X))
                             
    #Resampling
    if new_fs != fs:
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    return X,y,age,sex,fs

## Define the model (EEG Net)

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm

def EEGNet(nb_classes, Chans = 64, Samples = 128,
             dropoutRate = 0.5, kernLength = 64, F1 = 8,
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):

    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Chans, Samples, 1))

    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   name='Conv2D_1',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                   name='Depth_wise_Conv2D_1',
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)

    block2       = SeparableConv2D(F2, (1, 16),
                                   name='Separable_Conv2D_1',
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)

    flatten      = Flatten(name = 'flatten')(block2)

    dense        = Dense(nb_classes, name = 'output',
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'out_activation')(dense)

    return Model(inputs=input1, outputs=softmax)

# Experiment

## Experiment configuration 

In [6]:
import os

seed=23
folds=5
epochs_train = 500

save_folder = 'EEGNet_SDep_Motor_Training'

n_subjects = 51

PATH = f'{os. getcwd()}/{save_folder}'

## Run experiment

In [8]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from scipy.stats import ks_2samp
from scipy.spatial.distance import squareform

tf.random.set_seed(seed)

accs_dict = {}

subjects = np.arange(n_subjects)+1

db = '../input/brain-mediators-of-pain-motor/'

num_class = 2

load_args = dict(db = db,
            f_bank = np.asarray([[4., 60.]]),
            vwt = np.asarray([[0.5,2.5]]),
            new_fs = 256.0)

load_args['sbj'] = 1 

X_train, y_train, age, sex, fs = load_PAIN(**load_args)

Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


In [9]:
print("Starting experiment...\n")

t=time()
for sbj in subjects:
    if sbj != 18:
        print(f"\n\nSubject: {sbj}")
        load_args['sbj'] = sbj 

        X_train, y_train,age,sex,fs = load_PAIN(**load_args)

        X_train = X_train[..., np.newaxis]
        Y_train = tf.keras.utils.to_categorical(y_train,num_classes=num_class)
        # ----build model
        clf = KerasClassifier(
            EEGNet,
            random_state=seed,

            # ----model hyperparameters
            nb_classes=num_class, 
            Chans = X_train.shape[1], 
            Samples = X_train.shape[2],
            dropoutRate = 0.5,
            kernLength = X_train.shape[2],
            F1 = 4, D = 4, F2 = 32,

            # ----model config
            verbose=0,
            batch_size=500, #full batch        
            loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer="adam",
            optimizer_learning__rate=0.1,
            metrics = ['accuracy'],
            epochs = epochs_train
        )
        # ----search params
        param_grid =  {
                    'F1':[4,8],
                    'kernLength':[64, 128],
                    }

        # ----Gridsearch
        scoring = {"AUC": 'roc_auc', "Accuracy": make_scorer(accuracy_score),'Kappa':make_scorer(kappa)}

        cv = GridSearchCV(clf,param_grid,cv=StratifiedShuffleSplit(n_splits = folds, test_size = 0.2, random_state = seed),
                             verbose=0,n_jobs=1, scoring=scoring, refit="Accuracy")
        # ----find best params with gridsearch
        cv.fit(X_train,Y_train)

        # ----best score
        print('Subject',sbj,'Accuracy',cv.best_score_,'elapsed time',time()-t)
        print('---------')

        # --save acc to dict
        accs_dict[f'Subject{sbj}'] = cv.best_score_

        cv.cv_results_['best_index_'] = cv.best_index_

        full_path = os.path.join(PATH)

        try:
            os.makedirs(full_path)
        except:
            pass

        cv.best_estimator_.model_.save_weights(full_path + f'/Subject{sbj}_weights.h5')
        with open(full_path + f'/Subject{sbj}.p','wb') as f:
            pickle.dump(cv.cv_results_,f)         

Starting experiment...



Subject: 1
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


2024-10-28 13:53:20.455677: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 13:53:38.966355: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 13:53:53.722494: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 13:54:08.549438: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/dropout_6

Subject 1 Accuracy 0.8 elapsed time 346.3735041618347
---------


Subject: 2
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


2024-10-28 13:59:04.988339: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_21/dropout_42/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 13:59:19.744905: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_22/dropout_44/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 13:59:34.710139: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_23/dropout_46/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 13:59:49.563141: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2

Subject 2 Accuracy 0.8 elapsed time 690.0276777744293
---------


Subject: 3
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


...

Subject 49 Accuracy 0.675 elapsed time 16406.305109977722
---------


Subject: 50
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


2024-10-28 18:26:45.274385: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1008/dropout_2016/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 18:27:00.309576: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1009/dropout_2018/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 18:27:15.254645: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1010/dropout_2020/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 18:27:30.207500: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin sha

Subject 50 Accuracy 0.65 elapsed time 16750.923869371414
---------


Subject: 51
Not setting metadata
40 matching events found
No baseline correction applied
0 projection items activated


2024-10-28 18:32:29.668001: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1029/dropout_2058/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 18:32:44.413972: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1030/dropout_2060/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 18:32:59.123755: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1031/dropout_2062/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-10-28 18:33:14.857014: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin sha

Subject 51 Accuracy 0.65 elapsed time 17093.481481075287
---------


In [10]:
import shutil
shutil.make_archive("EEGNet_SDep_Motor_Training", 'zip', "/kaggle/working/EEGNet_SDep_Motor_Training")

'/kaggle/working/MotorConditionEEGNet_60Hz.zip'